#Recurrent Network for Character Prediction

In [1]:
from numpy import *
import caffe
from caffe import Net, NetSpec, layers as Lr, params as Pr
from caffe.proto.caffe_pb2 import NetParameter, SolverParameter
from caffe.proto.caffe_pb2 import BlobShape, EltwiseParameter
import string, os, cPickle as pickle
from glob import glob

In [2]:
save_pkl = lambda f, obj: pickle.dump(obj, open(f, 'wb'), protocol=-1)
load_pkl = lambda f: pickle.load(open(f, 'rb'))
PROD = EltwiseParameter.PROD
sf = lambda *x: string.join([str(i) for i in x], '_')

In [3]:
hypes = {
    'sequence_length': 100,
    'layers_num': 2,
    'state_dim': 2048,
    'batch_size': 64,
    'recurrent_unit': 'lstm' # rnn or lstm
}

use_gpu = True

txt_file = 'rawtxt/linux_kernel.txt'

T = hypes['sequence_length']
L = hypes['layers_num']
d = hypes['state_dim']
b = hypes['batch_size']

##Load Data

In [4]:
txt = open(txt_file, 'r').read()
C = sorted(set(txt))
n, k = len(txt), len(C)
if k <= 2**8: uintn = uint8
else: uintn = uint16
X = array([C.index(c) for c in txt], dtype=uintn)
Y = X[1:].copy()
X = X[:-1]

def chop(x, n=None, m=None):
    if n: m = len(x)//n
    if m: n = len(x)//m
    X = split(array(x[:m*n]), n)
    return array(X)

rshape = lambda A: chop(rollaxis(chop(A,n=b),1,0),m=T)
X, Y = map(rshape, [X, Y])

save('X.npy', X)
save('Y.npy', Y)
print X.shape

(969, 100, 64)


##Create Model

In [5]:
def rnn_step(h, h_below, kwargs_fc):
    
    kwargs_fc['num_output'] = d
    
    h = Lr.Concat(h_below, h)
    h = Lr.InnerProduct(h, **kwargs_fc)
    h = Lr.TanH(h)
    
    return h

In [6]:
def lstm_step(h, h_below, kwargs_fc):
    
    kwargs_fc['num_output'] = 2*d
    
    c, h = Lr.Slice(h, slice_point=d/2, ntop=2)
    h = Lr.Concat(h_below, h)
    h = Lr.InnerProduct(h, **kwargs_fc)
    i, f, o, g = Lr.Slice(h, slice_point=[d/2,d,3*d/2], ntop=4)
    i, f, o = map(Lr.Sigmoid, [i, f, o])
    g = Lr.TanH(g)
    c = Lr.Eltwise(Lr.Eltwise(f, c, operation=PROD), Lr.Eltwise(i, g, operation=PROD))
    h = Lr.Eltwise(o, Lr.TanH(c), operation=PROD)
    h = Lr.Concat(c, h)
    
    return h

In [7]:
def get_net_param(T, b, drop=True):
    
    net_spec = NetSpec()

    bsX, bsY, bsH = [BlobShape() for i in range(3)]
    bsH.dim.extend([b,d])
    bsX.dim.extend([b,k])
    bsY.dim.extend([b])

    if hypes['recurrent_unit'] == 'rnn': step = rnn_step
    if hypes['recurrent_unit'] == 'lstm': step = lstm_step
    
    get_kwargs_fc = lambda t, l: {
        'param': [{'lr_mult': 1, 'decay_mult': 1, 'name': sf('W', l)},
                  {'lr_mult': 2, 'decay_mult': 0, 'name': sf('b', l)}],
        'weight_filler': {'type': 'uniform', 'min': -0.01, 'max': 0.01},
        'name': sf('fc', t, l)
    }
    
    h = []
    for l in range(L):
        h.append(Lr.DummyData(shape=bsH))
        setattr(net_spec, sf('h',0,l), h[l])
    
    for t in range(T):

        x = Lr.DummyData(shape=bsX)
        y = Lr.DummyData(shape=bsY)

        h[0] = step(h[0], x, get_kwargs_fc(t, 0))
        for l in range(1, L):
            h[l] = step(h[l], h[l-1], get_kwargs_fc(t, l))
            if drop: h[l] = Lr.Dropout(h[l])

        kwargs_fc = get_kwargs_fc(t, L)
        kwargs_fc['num_output'] = k
        
        z = Lr.InnerProduct(h[-1], **kwargs_fc)
        loss = Lr.SoftmaxWithLoss(z, y)

        setattr(net_spec, sf('x', t), x)
        setattr(net_spec, sf('y', t), y)
        setattr(net_spec, sf('z', t), z)
        for l in range(L): setattr(net_spec, sf('h', t+1, l), h[l])
        setattr(net_spec, sf('loss', t), loss)

    return net_spec.to_proto()

In [8]:
solver_hypes = {
    'base_lr': 1e-3,
    'weight_decay': 1e-4,
    'lr_policy': 'exp',
    'gamma': 0.9999,
    'clip_gradients': 100,
    'solver_type': SolverParameter.RMSPROP,
    'rms_decay': 0.8
}

if use_gpu:
    solver_hypes['solver_mode'] = SolverParameter.GPU
    solver_hypes['device_id'] = 0
else:
    solver_hypes['solver_mode'] = SolverParameter.CPU

solver_param = SolverParameter()
solver_param.net_param.CopyFrom(get_net_param(T, b))
solver_param.test_net_param.add()
solver_param.test_net_param[0].CopyFrom(get_net_param(T, b, drop=False))
solver_param.test_iter.extend([1])
solver_param.test_interval = 10**9
for pr, val in solver_hypes.iteritems():
    setattr(solver_param, pr, val)

with open('solver.prototxt', 'w') as f: f.write(str(solver_param))

###Better to run this from the command line

In [ ]:
!python train_recurrent.py

##Deploy (run this while training)

In [13]:
seed_chars = '#inc'
temperature = 0.7
num_chars = 1000

params = load_pkl(sorted(glob('params/iter*.pkl'))[-1])
open('deploy.prototxt', 'w').write(str(get_net_param(1, 1, drop=False)))
net = caffe.Net('deploy.prototxt', 1)

param_corresp = [(sf('fc',l), sf('fc',0,l)) for l in range(L+1)]
for ki, kj in param_corresp:
    for ip, p in enumerate(net.params[kj]):
        p.data[...] = params[ki][ip]

for c in seed_chars:
    x = C.index(c)
    net.blobs[sf('x',0)].data[...] = 0
    net.blobs[sf('x',0)].data[0, x] = 1
    for l in range(L):
        state_i = net.blobs[sf('h',0,l)].data
        state_f = net.blobs[sf('h',1,l)].data
        state_i[...] = state_f
    net.forward()

gen_chars = []
for t in range(num_chars):
    z = net.blobs[sf('z',0)].data[0].copy().astype(float)
    p = (lambda x: x/sum(x))(exp(z/temperature))
    x = random.choice(range(k), p=p)
    gen_chars.append(C[x])
    net.blobs[sf('x',0)].data[...] = 0
    net.blobs[sf('x',0)].data[0, x] = 1
    for l in range(L):
        state_i = net.blobs[sf('h',0,l)].data
        state_f = net.blobs[sf('h',1,l)].data
        state_i[...] = state_f
    net.forward()
            
print seed_chars + string.join(gen_chars, '')

#incled_func int nest_timer_get_task_struct argid = posix_cpu_torif_exp_rep(= 'l'-- clase();
			ret = committing_get_task(struct trace_parser_cpu(t, unsigned long)addr)
{
	return ret;
}

#ifdef CONFIG_EVER_TRACERD) {
			if (strcmp(nr))->pend);
	end->struct irq_desc *desc) {
			if (modit(&q->endingf_panse->pid, desc);
	return 0;
			ret = AUDIT_INLAD_DIVE);
			ret = batance_prio(poid_get, profile_hash(struct second_name))
		return -EINVAL;
	int entryl = true;

		ret = posix_probe(page);
				if (!in_timersnoring_flags);
	return 1;
				create_continue;
			continue;
			ns = psentity_before(&cpu_buffee->committin = entrysnot);
		}

			if (second_neg_size());
			ret = pred;
		char = prefix_lock);
			if (it == '\0') {
				drn = -EINVAL;
		ret = prid;
		}
			return 0;
			can = ((calt->_lage);
	INIT_REC);
extern 0;
			ret = -EINVAL;
		ret = pagentry_register_nettime = FPRACE_LEG(note)->prefix_name))
		return 1;
			if (strlcp((dl.callb_PR_PROER);
			return ret;
}

static int return && !  = pare =